In [24]:
import os
import base64
import pandas as pd
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [25]:
# Functions
def get_doc_content(service, document_id):
    document = service.documents().get(documentId=document_id).execute()
    doc_content = document.get('body').get('content')
    return doc_content

# Function to extract subject and body from doc content
def extract_subject_and_body(doc_content):
    subject = ''
    body = ''
    is_subject = False
    is_body = False
    for item in doc_content:
        if 'paragraph' in item:
            elements = item.get('paragraph').get('elements')
            text = ''
            for element in elements:
                text_run = element.get('textRun', {})
                content = text_run.get('content', '')
                text += content

            if text.strip() == 'email_subject':
                is_subject = True
                continue
            elif text.strip() == 'email_body':
                is_body = True
                is_subject = False
                continue

            if is_subject:
                subject += text
            elif is_body:
                body += text

    return subject.strip(), body.strip()

In [26]:
# Establish GoogleAPI Scopes and Instance
SCOPES = ['https://www.googleapis.com/auth/gmail.compose', 'https://www.googleapis.com/auth/documents.readonly']
creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('client_secret_600207218247-88idqn5h1tadq1d2gl7tkn826l63ud41.apps.googleusercontent.com.json', SCOPES)
        creds = flow.run_local_server(port=0)
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=600207218247-88idqn5h1tadq1d2gl7tkn826l63ud41.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A53846%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.compose+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocuments.readonly&state=EbGPOxK6ol89O7B64K5x59uHi1t1vL&access_type=offline


In [27]:
# Extract email content from Google Doc
doc_service = build('docs', 'v1', credentials=creds)
# The document id is found between the /d and the /edit portions of the GoogleDoc URL 
# Here is an example URL https://docs.google.com/document/d/13_xSd2YaiEP_p1DJl0A5gnsg2z0ahaN27BAtXfWcw2M/edit
document_id = '13_xSd2YaiEP_p1DJl0A5gnsg2z0ahaN27BAtXfWcw2M'
doc_content = get_doc_content(doc_service, document_id)
email_subject, email_body = extract_subject_and_body(doc_content)

# Read names and emails from CSV file
unmatched_names = pd.read_csv('unmatched_names.csv')

# Sending personalized emails
for index, row in unmatched_names.iterrows():
    recipient_email = row["Email"]  # Extract email from the "Email" column
    personalized_email_body = email_body.replace("{name}", row["First"] + " " + row["Last"])
    message = f'To: {recipient_email}\nSubject: {email_subject}\n\n{personalized_email_body}'
    message_bytes = message.encode('utf-8')
    encoded_message = base64.urlsafe_b64encode(message_bytes).decode('utf-8')

    email = {'raw': encoded_message}

    # Try to send the email, print error content if there is an HttpError
    try:
        gmail_service.users().messages().send(userId='me', body=email).execute()
    except HttpError as error:
        print(error.content)